<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/OST_to_Online_Archive_Migrator3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox

class EmailMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'source_folder_path': 'N/A',
            'destination_store_name': 'N/A',
            'destination_root_path': 'N/A',
            'folders_processed': 0,
            'items_moved': 0,
            'items_failed': 0,
            'folder_details': {}
        }

    def setup_logging(self):
        """Setup comprehensive logging for the migration."""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'ost_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)

        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger().addHandler(file_handler)

        logging.info(f"Logging to: {log_filename}")

    def find_folder_by_path(self, namespace, folder_path):
        """Find an Outlook folder by its full path."""
        try:
            # Get the root of the "Personal Folders" or "Mailbox"
            # It's better to navigate from the root of the default store
            root_folder = namespace.GetDefaultFolder(6).Parent

            folders = folder_path.split('\\')
            current_folder = root_folder
            for folder_name in folders:
                if folder_name in [f.Name for f in current_folder.Folders]:
                    current_folder = current_folder.Folders[folder_name]
                else:
                    logging.error(f"Source folder not found: '{folder_path}'")
                    return None
            return current_folder
        except Exception as e:
            logging.error(f"Error finding folder '{folder_path}': {e}", exc_info=True)
            messagebox.showerror("Error", f"Could not find the source folder: {e}")
            return None

    def create_folder_if_not_exists(self, parent_folder, folder_name):
        """Creates a new folder in the parent folder if it doesn't already exist."""
        try:
            return parent_folder.Folders[folder_name]
        except Exception:
            try:
                new_folder = parent_folder.Folders.Add(folder_name)
                logging.info(f"Created new folder: {new_folder.FolderPath}")
                return new_folder
            except Exception as e:
                logging.error(f"Failed to create folder '{folder_name}': {e}", exc_info=True)
                return None

    def migrate_folder_contents(self, source_folder, destination_folder):
        """Moves all items from a source folder to a destination folder."""
        moved_count = 0
        failed_count = 0

        try:
            items_list = list(source_folder.Items)
            for item in items_list:
                try:
                    item.Move(destination_folder)
                    moved_count += 1
                    self.migration_report['items_moved'] += 1
                except Exception as e:
                    failed_count += 1
                    self.migration_report['items_failed'] += 1
                    logging.warning(f"Failed to move item '{getattr(item, 'Subject', 'N/A')}' from '{source_folder.FolderPath}': {e}")
        except Exception as e:
            logging.error(f"Error accessing items in folder '{source_folder.FolderPath}': {e}", exc_info=True)
            failed_count += len(items_list) if 'items_list' in locals() else 0

        return moved_count, failed_count

    def migrate_folders_recursively(self, source_folder, destination_parent_folder):
        """Recursively migrates subfolders and their contents."""
        self.migration_report['folders_processed'] += 1

        # Create the corresponding folder in the destination
        destination_folder = self.create_folder_if_not_exists(destination_parent_folder, source_folder.Name)
        if not destination_folder:
            logging.error(f"Failed to create destination folder for '{source_folder.Name}'. Skipping contents.")
            self.migration_report['folder_details'][source_folder.FolderPath] = {
                'moved_items': 0, 'failed_items': len(source_folder.Items) if hasattr(source_folder, 'Items') else 0
            }
            return

        # Move all items from the source to the new destination folder
        moved_count, failed_count = self.migrate_folder_contents(source_folder, destination_folder)

        self.migration_report['folder_details'][source_folder.FolderPath] = {
            'moved_items': moved_count,
            'failed_items': failed_count,
            'total_items_in_source': moved_count + failed_count
        }

        logging.info(f"Processed folder '{source_folder.Name}': {moved_count} moved, {failed_count} failed.")

        # Recursively call this function for all subfolders
        for subfolder in source_folder.Folders:
            self.migrate_folders_recursively(subfolder, destination_folder)

    def run_migration(self):
        """Main function to run the migration process."""
        self.migration_report['start_time'] = datetime.now().isoformat()

        pythoncom.CoInitialize()
        outlook = None

        # --- Configuration ---
        SOURCE_FOLDER_PATH = "Inbox\\Trade&BO\\Trade"
        DESTINATION_STORE_NAME = "Online Archive - user@domain.com"

        try:
            outlook = win32com.client.Dispatch("Outlook.Application")
            namespace = outlook.GetNamespace("MAPI")

            # --- Step 1: Find the source folder ---
            source_folder = self.find_folder_by_path(namespace, SOURCE_FOLDER_PATH)
            if not source_folder:
                logging.error("Source folder not found. Exiting.")
                return False

            self.migration_report['source_folder_path'] = source_folder.FolderPath

            # --- Step 2: Select the destination store (hardcoded) ---
            target_root_folder = None
            for store in namespace.Stores:
                if store.DisplayName == DESTINATION_STORE_NAME:
                    target_root_folder = store.GetRootFolder()
                    break

            if not target_root_folder:
                logging.error(f"Destination store '{DESTINATION_STORE_NAME}' not found. Please ensure it is open in Outlook.")
                messagebox.showerror("Error", f"Destination store '{DESTINATION_STORE_NAME}' not found.")
                return False

            self.migration_report['destination_store_name'] = DESTINATION_STORE_NAME
            self.migration_report['destination_root_path'] = target_root_folder.FolderPath

            print("\n" + "=" * 50)
            print("CONFIRMATION: You are about to MOVE emails from:")
            print(f"Source: {self.migration_report['source_folder_path']}")
            print(f"To Destination: {self.migration_report['destination_root_path']} ({self.migration_report['destination_store_name']})")
            print("\nThis action will MOVE items and CANNOT BE UNDONE.")
            print("Please ensure you have backups of your data.")
            print("=" * 50)

            confirm = input("Type 'CONFIRM' to proceed with the migration: ")
            if confirm.strip().upper() != 'CONFIRM':
                logging.warning("Migration cancelled by user.")
                print("Operation cancelled.")
                return False

            # --- New logic to create the full destination path before migrating ---
            path_parts = SOURCE_FOLDER_PATH.split('\\')
            current_dest_folder = target_root_folder
            for part in path_parts:
                current_dest_folder = self.create_folder_if_not_exists(current_dest_folder, part)
                if not current_dest_folder:
                    logging.error(f"Failed to create the full destination path. Aborting migration.")
                    return False

            # --- Step 3: Start the recursive migration to the final, created folder ---
            self.migration_report['folders_processed'] += 1
            moved_count, failed_count = self.migrate_folder_contents(source_folder, current_dest_folder)

            self.migration_report['folder_details'][source_folder.FolderPath] = {
                'moved_items': moved_count,
                'failed_items': failed_count,
                'total_items_in_source': moved_count + failed_count
            }
            logging.info(f"Processed folder '{source_folder.Name}': {moved_count} moved, {failed_count} failed.")

            for subfolder in source_folder.Folders:
                self.migrate_folders_recursively(subfolder, current_dest_folder)

            # --- Step 4: Finalize and report ---
            self.migration_report['end_time'] = datetime.now().isoformat()
            self.generate_report()

            if self.migration_report['items_failed'] == 0:
                logging.info("Migration completed successfully!")
                return True
            else:
                logging.warning(f"Migration completed with {self.migration_report['items_failed']} errors. Check the log file for details.")
                return False

        except Exception as e:
            logging.error(f"Critical error during migration: {e}", exc_info=True)
            messagebox.showerror("Migration Error", f"A critical error occurred. Check logs for details: {e}")
            return False
        finally:
            if outlook:
                del outlook
            pythoncom.CoUninitialize()

    def generate_report(self):
        """Generates a comprehensive report of the migration."""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        print("\n=== MIGRATION SUMMARY ===")
        print(f"Source Folder: {self.migration_report['source_folder_path']}")
        print(f"Destination: {self.migration_report['destination_root_path']} ({self.migration_report['destination_store_name']})")
        print(f"Folders Processed: {self.migration_report['folders_processed']}")
        print(f"Total Items Moved: {self.migration_report['items_moved']}")
        print(f"Total Items Failed: {self.migration_report['items_failed']}")

        if self.migration_report['items_failed'] > 0:
            print("\n--- Failed Items Summary ---")
            for folder, details in self.migration_report['folder_details'].items():
                if details['failed_items'] > 0:
                    print(f"  - Folder '{folder}': {details['failed_items']} failed items")
            print("Check the log file for specific details on each failed item.")

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("  OST to Online Archive Email Migrator (with structure)   ")
    print("=" * 60)
    print("This tool will find a specific OST folder and move its contents")
    print("to a destination of your choice, preserving the folder structure.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    migrator.run_migration()